# A YouTube Video Summarizer Using Whisper and LangChain

• Find the  [Notebook](https://colab.research.google.com/github/towardsai/ragbook-notebooks/blob/main/notebooks/Chapter%2007%20-%20Create_a_YouTube_Video_Summarizer_Using_Whisper_and_LangChain_.ipynb)  for this section at  [towardsai.net/book](http://towardsai.net/book).

The project involves a series of steps, starting with downloading the audio file from YouTube. Once the audio file is obtained, it is transcribed using **Whisper**. After the transcription is complete, the text is summarized using LangChain, employing three different approaches: *stuff*, *refine*, and *map_reduce*. Finally, multiple transcriptions are added to the DeepLake database to enable question-answering for those videos.

The following diagram explains what we are going to do in this project:

![image](./youtube_video_summarizer.jpg)

*Our YouTube video summarizer pipeline.*

As usual, start by installing the packages using the command: `!pip install langchain==0.0.208 deeplake openai==0.27.8 tiktoken,  yt_dlp, and openai-whisper.`

Next, install [ffmpeg](https://ffmpeg.org/); it is a prerequisite for the ***yt_dlp*** package.

Next, add the API key for OpenAI and Deep Lake services to the environment variables.

In [ ]:
import os
from langchain_custom_utils.helper import get_openai_api_key, get_deeplake_api_key, print_response
OPENAI_API_KEY = get_openai_api_key()
DEEPLAKE_API_KEY = get_deeplake_api_key()

The tutorial teaches how to programmatically summarize a video featuring Yann LeCun, a notable computer scientist and AI researcher. The video covers LeCun’s thoughts on the challenges associated with large language models. However, the code would work with any other video as long as it can be summarized using only its audio (as the model won’t know what is shown in the video) and that ideally contains only a few speakers. Video podcasts are ideal for this project.

The download_mp4_from_youtube() function downloads the highest quality mp4 video file from a given YouTube link and saves it to a specified path and filename. To use this function, simply copy and paste the URL of the chosen video into it.

In [ ]:
import yt_dlp

def download_mp4_from_youtube(url):
    # Set the options for the download
    filename = 'lecuninterview.mp4'
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]',
        'outtmpl': filename,
        'quiet': True,
    }

    # Download the video file
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        result = ydl.extract_info(url, download=True)

url = "https://www.youtube.com/watch?v=mBjPyte2ZZo"
download_mp4_from_youtube(url)

Now that the video MP4 has been downloaded, the next step is to transcribe its audio using a speech-to-text model. One of the currently most popular open-source speech-to-text models is OpenAI’s Whisper.

## Transcribing Audio with Whisper

***Whisper*** is an advanced automatic speech recognition system developed by OpenAI. It’s trained on a dataset of 680,000 hours of multilingual and multitasking supervised data from the web. This extensive and diverse dataset contributes to the system’s ability to efficiently manage accents, background noise, and technical jargon.

The previously installed whisper package includes the `.load_model()` method, which downloads the model and transcribes a video file. Several models are available: tiny, base, small, medium, and large, for balancing accuracy and processing speed. We will use the 'base' model for this example.

In [ ]:
import whisper

model = whisper.load_model("base")
result = model.transcribe("lecuninterview.mp4")
print(result['text'])

> /home/cloudsuperadmin/.local/lib/python3.9/site-packages/whisper/transcribe.py:114:
> UserWarning: FP16 is not supported on CPU; using FP32 instead  
> warnings.warn("FP16 is not supported on CPU; using FP32 instead")     
> Hi, I'm Craig Smith, and this is I on A On. This week I talked to Jan
> LeCoon, one of the seminal figures in deep learning development and a
> long-time proponent of self-supervised learning. Jan spoke about
> what's missing in large language models and his new joint embedding
> predictive architecture which may be a step toward filling that gap.
> He also talked about his theory of consciousness and the potential for
> AI systems to someday exhibit the features of consciousness. It's a
> fascinating conversation that I hope you'll enjoy. Okay, so Jan, it's
> great to see you again. I wanted to talk to you about where you've
> gone with so supervised learning since last week's spoke. In
> particular, I'm interested in how it relates to large language models
> because they have really come on stream since we spoke. In fact, in
> your talk about JEPA, which is joint embedding predictive
> architecture. […and so on]

The result is generated as raw text and can be saved to a text file.

In [ ]:
with open ('text.txt', 'w') as file:  
    file.write(result['text'])

Once the transcription is ready, the next step is to split it into chunks using a text splitter and then use the chunks to generate a summary.

### Splitting Text and Generating the Summary

Import the necessary classes and utilities from the LangChain library.

In [ ]:
from langchain import OpenAI, LLMChain
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain

llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0)

The code below creates an instance of the `RecursiveCharacterTextSplitter` class. This class is used to split input text into more manageable, smaller segments.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=0, separators=[" ", ",", "\n"]
)

The `RecursiveCharacterTextSplitter` is set up with a `chunk_size` of 1000 characters, without any `chunk_overlap`, and uses spaces, commas, and newline characters as separators.

Now, open the previously saved text file and use the `.split_text()` method to segment the transcripts.

In [ ]:
from langchain.docstore.document import Document

with open('text.txt') as f:
    text = f.read()

texts = text_splitter.split_text(text)
docs = [Document(page_content=t) for t in texts[:4]]

Each Document object is initialized with the content of a chunk from the texts list. The `[:4]` slice notation indicates that only the first four chunks will be used to create the Document objects.

In [ ]:
from langchain.chains.summarize import load_summarize_chain
import textwrap

chain = load_summarize_chain(llm, chain_type="map_reduce")

output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)

> Craig Smith interviews Jan LeCoon, a deep learning developer and
> proponent of self-supervised learning, about his new joint embedding
> predictive architecture and his theory of consciousness. Jan's
> research focuses on self-supervised learning and its use for
> pre-training transformer architectures, which are used to predict
> missing words in a piece of text. Additionally, large language models
> are used to predict the next word in a sentence, but it is difficult
> to represent uncertain predictions when applying this to video.

>💡The textwrap library in Python provides a convenient way to wrap and format plain text by adjusting line breaks in an input paragraph. It is particularly useful when displaying text within a limited width, such as in console outputs, emails, or other formatted text displays. The library includes convenience functions like `wrap`, `fill`, and `shorten`, as well as the `TextWrapper` class that handles most of the work. If you’re curious, find more information on [Text wrapping and filling](https://docs.python.org/3/library/textwrap.html) at [towardsai.net/book](http://towardsai.net/book).

The following code shows the prompt template used with the `map_reduce` chain type. The `map-reduce` process first summarizes each document separately using a language model (Map step), turning each into a new document. Then, it combines all of them into one document (Reduce step) to form the final summary.

In [ ]:
print(chain.llm_chain.prompt.template)

>Write a concise summary of the following:\n\n\n"{text}"\n\n\n CONCISE SUMMARY:

The "stuff" approach involves using all text from the transcribed video in a single prompt, which is a basic and straightforward method. However, it might not be the most efficient for handling large volumes of text, and it may overflow the maximum context size of the LLM. It’s always advised to monitor the number of tokens used in the prompts because different LLM APIs may behave differently when the maximum token limit is exceeded. Some APIs may throw errors, while others may silently use only the first part of the prompt until it fills the LLM context size completely, thus generating an output with an incomplete prompt.

To generate the summary for this tutorial, we’re going to experiment with the prompt below, which will output the summary as bullet points:

In [ ]:
prompt_template = """Write a concise bullet point summary of the following:

{text}

CONSCISE SUMMARY IN BULLET POINTS:"""

BULLET_POINT_PROMPT = PromptTemplate(template=prompt_template,
                        input_variables=["text"])

We also initialized the summarization chain using the stuff as `chain_type` and the prompt above:

In [ ]:
chain = load_summarize_chain(llm,
                             chain_type="stuff",
                             prompt=BULLET_POINT_PROMPT)

output_summary = chain.run(docs)

wrapped_text = textwrap.fill(output_summary,
                             width=1000,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)

> - Jan LeCoon is a seminal figure in deep learning development and a long time proponent of self-supervised learning  
> - Discussed his new joint embedding predictive architecture which may be a step toward filling the gap in large language models  
> - Theory of consciousness and potential for AI systems to exhibit features of consciousness  
> - Self-supervised learning revolutionized natural language processing  
> - Large language models lack a world model and are generative models, making it difficult to represent uncertain predictions

**LangChain** provides the flexibility to create custom prompts tailored to specific needs. For instance, if the objective is to receive a summarization output in French, one can construct a prompt instructing the language model to generate a summary in French.

The 'refine' summarization chain is an approach designed to generate more precise and *context-sensitive summaries*. This method follows an iterative process to enhance the summary by incorporating additional context as needed. In practice, it initiates by summarizing the first text chunk. Subsequently, the generated summary is enriched with new information from each subsequent chunk. It can produce more accurate and context-aware summaries than chains like '*stuff*' and '*map_reduce*', at the cost of more LLM calls.

In [ ]:
chain = load_summarize_chain(llm, chain_type="refine")
output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)

>Craig Smith interviews Jan LeCoon, a deep learning developer and proponent of self-supervised learning, about his new joint embedding predictive architecture and his theory of consciousness. Jan discusses the gap in large language models and the potential for AI systems to exhibit features of consciousness. He explains how self-supervised learning has revolutionized natural language processing through the use of transformer architectures for pre-training, such as taking a piece of text, removing some of the words, and replacing them with black markers to train a large neural net to predict the words that are missing. This technique has been used in practical applications such as contact moderation systems on Facebook, Google, YouTube, and more. Jan also explains how this technique can be used to represent uncertain predictions in generative models, such as predicting the missing words in a text, or predicting the missing frames in a video.